In [18]:
# Library imports

import os
import pandas as pd
from nltk import sent_tokenize
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import string, unicodedata, re
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from collections import defaultdict
import pickle

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
# Define the file path
file_path_creole_law = './/content//ley_wayuu.txt'  # Replace with the path to your text file

# Read the content of the file and split it into sentences
with open(file_path_creole_law, 'r', encoding='utf-8') as file:
    sentences_wayuu = [line.strip() for line in file if line.strip()]

In [29]:
len(sentences_wayuu)

42

In [30]:
# Define the file path
file_path_spanish_law = './/content//ley_espanol.txt'  # Replace with the path to your text file

# Read the content of the file and split it into sentences
with open(file_path_spanish_law, 'r', encoding='utf-8') as file:
    sentences_spanish = [line.strip() for line in file if line.strip()]

In [31]:
len(sentences_spanish)

42

In [35]:
df_wayuu = pd.DataFrame({'español': sentences_spanish, 'wayuu': sentences_wayuu})

In [36]:
df_wayuu_prepros = text_preprocessing(df_wayuu, 'español', 'wayuu')

In [38]:
df_wayuu_prepros[10:15]

,español,wayuu
10,<s> pobreza </s>,<s> moju kat muli akat </s>
11,<s> camara de representante </s>,<s> shikipu jana tuu </s>
12,<s> politico </s>,<s> aluuwatawaa </s>
13,<s> construir </s>,<s> akumaja </s>
14,<s> acordado </s>,<s> oonowaa amaa </s>


In [33]:
from nltk.corpus.reader.tagged import word_tokenize
def remove_non_ascii_from_text(text):
  # Remove non-ASCII characters from a text string
  new_text = []
  words = word_tokenize(text, language='english')
  for word in words:
    new_word = unicodedata.normalize('NFKD', str(word)).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    new_text.append(new_word)
  new_text = ' '.join(new_text)
  return new_text

def to_lowercase_from_text(text):
  # Convert all characters to lowercase from a text string
  new_text = text.lower()
  return new_text

def remove_punctuation_from_text(text):
  # Remove punctuation from a text string
  new_text = re.sub(r'[^\w\s?!]', '', text)
  return new_text

def add_tags(text):
  return '<s> ' + text + ' </s>'

def text_preprocessing(df, column_name_1, column_name_2):
  # Apply the preprocessing functions to a specific column in the DataFrame
  df[column_name_1] = df[column_name_1].apply(remove_non_ascii_from_text)
  df[column_name_1] = df[column_name_1].apply(to_lowercase_from_text)
  df[column_name_1] = df[column_name_1].apply(remove_punctuation_from_text)
  df[column_name_1] = df[column_name_1].apply(add_tags)
  df[column_name_2] = df[column_name_2].apply(remove_non_ascii_from_text)
  df[column_name_2] = df[column_name_2].apply(to_lowercase_from_text)
  df[column_name_2] = df[column_name_2].apply(remove_punctuation_from_text)
  df[column_name_2] = df[column_name_2].apply(add_tags)

  return df

In [3]:
import os

directory_path = './/content//biblia_espanol//'

all_file_text_spanish = []

# Function to extract the numerical part of a filename
def extract_file_number(filename):
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    return 0

# Get a list of files in the directory and sort them by their numerical part
files_in_directory = os.listdir(directory_path)
files_in_directory = sorted(files_in_directory, key=lambda x: extract_file_number(x))

for filename in files_in_directory:
    if filename.startswith('biblia_espanol_') and filename.endswith('.txt'):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            start_index = text.find("\n1")
            file_text = text[start_index + len("\n1"):]
            file_text = file_text.replace("\n", " ")
            file_text = ' '.join(file_text.split())
            file_text = re.sub(r'[^\w\s]', '', file_text)
            file_text = unicodedata.normalize('NFKD', file_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            file_text = re.sub(r'(\d+)', r'.', file_text)
            file_text = re.sub(r' (?=\.)', '', file_text)
            file_text = file_text.replace("..", ".")
            file_text = re.sub(r'(\.)', r'. ', file_text)
            file_text = file_text.split('. ')
            file_text = [text for text in file_text if text != ""]
            all_file_text_spanish += file_text

In [4]:
directory_path = './/content//biblia_wayuu/'

all_file_text_wayuu = []

# Function to extract the numerical part of a filename
def extract_file_number(filename):
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    return 0

# Get a list of files in the directory and sort them by their numerical part
files_in_directory = os.listdir(directory_path)
files_in_directory = sorted(files_in_directory, key=lambda x: extract_file_number(x))

for filename in files_in_directory:
    if filename.startswith('biblia_wayuu_') and filename.endswith('.txt'):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            start_index = text.find("\n1")
            file_text = text[start_index + len("\n1"):]
            file_text = file_text.replace("\n", " ")
            file_text = ' '.join(file_text.split())
            file_text = re.sub(r'[^\w\s]', '', file_text)
            file_text = unicodedata.normalize('NFKD', file_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            file_text = re.sub(r'(\d+)', r'.', file_text)
            file_text = re.sub(r' (?=\.)', '', file_text)
            file_text = file_text.replace("..", ".")
            file_text = re.sub(r'(\.)', r'. ', file_text)
            file_text = file_text.split('. ')
            file_text = [text for text in file_text if text != ""]
            all_file_text_wayuu += file_text

In [5]:
len(all_file_text_wayuu), len(all_file_text_spanish)

(1292, 1292)

In [6]:
bible_df = pd.DataFrame({'español': all_file_text_spanish, 'wayuu': all_file_text_wayuu})

In [7]:
ls_sp = []
ls_wa = []
for ind in range(len(bible_df["español"])):
    l = len(bible_df["español"][ind])
    ls_sp.append(l)
    l = len(bible_df["wayuu"][ind])
    ls_wa.append(l)

In [8]:
iii = np.argmax(ls_sp)

In [9]:
bible_df["español"][iii]

'Asi que le enviaron algunos de sus propios seguidores junto con otros que pertenecian al partido de Herodes para que le dijeran  Maestro sabemos que tu eres sincero y que ensenas con toda verdad a vivir como Dios quiere no te preocupa el que diran ni juzgas a la gente por las apariencias'

In [10]:
bible_df["wayuu"][iii]

'Najutakalaka na fariseokana wane na nekirajuinkana nunainmuin Jesus ooulaka wane numaajanakana Herodes chi sulaulashikai mma sunain namuin numuin Jesus Ekirajuikalee watujaa aauchi pia sunain shiimuin tu pumakat sunain ekirajaa tu nuluwataakat anain Maleiwa je sunain matujuin pia kachekalaa suutku wayuu'

In [11]:
bible_df_prepros = text_preprocessing(bible_df, 'español', 'wayuu')

In [12]:
bible_df_prepros

,español,wayuu
0,<s> cuando ya los sabios se habian ido un ange...,<s> shiasa suchikijee nounapa na wayuu wuinpuj...
1,<s> jose se levanto tomo al nino y a su madre ...,<s> nuchijiraakalaka jose sunain nujuittuin nu...
2,<s> donde estuvieron hasta que murio herodes e...,<s> eeinjanale kepiain naya waneereeya ouktapa...
3,<s> al darse cuenta herodes de que aquellos sa...,<s> je nutujaapa saau herodes sunain nemeerain...
4,<s> asi se cumplio lo escrito por el profeta j...,<s> alatusu tuu supula keraainjatuin tu nunuik...
...,...,...
1287,<s> y aunque es verdad que dios no ha tomado e...,<s> nnojotsu kasalajanapuuin sumuin wayuu nutu...
1288,<s> y ya tiene fijado el dia en que ha de juzg...,<s> suka jamuin maleiwakai niitaain wane kai n...
1289,<s> cuando oyeron hablar de resurreccion de mu...,<s> je nouluku na eejanakana naapapa suchiki t...
1290,<s> asi que pablo abandono la reunion </s>,<s> nuunakalaka pablo namaanajee </s>


In [39]:
wayuu = pd.concat([df_wayuu_prepros, bible_df_prepros], ignore_index=True)

In [40]:
wayuu

,español,wayuu
0,<s> principio </s>,<s> sutia sukuipa </s>
1,<s> acuerdo final para la terminacion del conf...,<s> koutushi naa wayukana napushuale supula ei...
2,<s> salvaguardas y garantia </s>,<s> akuipa aka kojutunjatuu wakuipa </s>
3,<s> reforma rural integral </s>,<s> awanajawa sukuipaa woumain </s>
4,<s> planes de vida </s>,<s> akumaja supula anan akuipaa wepiapaa </s>
...,...,...
1329,<s> y aunque es verdad que dios no ha tomado e...,<s> nnojotsu kasalajanapuuin sumuin wayuu nutu...
1330,<s> y ya tiene fijado el dia en que ha de juzg...,<s> suka jamuin maleiwakai niitaain wane kai n...
1331,<s> cuando oyeron hablar de resurreccion de mu...,<s> je nouluku na eejanakana naapapa suchiki t...
1332,<s> asi que pablo abandono la reunion </s>,<s> nuunakalaka pablo namaanajee </s>


In [41]:
wayuu.to_excel('wayuu_dataset.xlsx', index=False)

In [42]:
# Function to count unique words in a text column
def count_unique_words(text_column):
    # Tokenize the text into words
    words = text_column.str.split().explode()

    # Count unique words using a set
    unique_words = set(words)

    return len(unique_words)

# Apply the function to your DataFrame's text column
unique_word_count = count_unique_words(wayuu['español'])

print(f"Number of unique words: {unique_word_count}")

Number of unique words: 3877
